In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [12]:
train = pd.read_csv('p3_train.csv')
test = pd.read_csv('p3_test.csv')

In [13]:
X_train, X_test, y_train, y_test = train_test_split(train['text'], train['label'], test_size=0.2, random_state=42)
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [14]:
!git clone https://github.com/facebookresearch/fastText.git
!cd fastText
!pip install fastText

In [15]:
import fasttext.util
fasttext.util.download_model('en', if_exists='ignore')  # English
# ft = fasttext.load_model('cc.en.300.bin')

In [16]:
from gensim.models.fasttext import load_facebook_model

# Load FastText embeddings
ft_model = load_facebook_model('cc.en.300.bin')


In [17]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, Embedding, SimpleRNN, Dense, Dropout, Conv1D, MaxPooling1D, GlobalMaxPooling1D


# tokenization and padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
max_length = max([len(seq) for seq in X_train_seq])
max_length = max([len(seq) for seq in X_test_seq])
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

# RNN model
embedding_dim = 300
model = Sequential()
model.add(Embedding(input_dim=len(ft_model.wv.key_to_index)+1,
                    output_dim=embedding_dim,
                    input_length=max_length,
                    weights=[np.vstack((np.zeros((1, embedding_dim)), ft_model.wv.vectors))],
                    trainable=False))
model.add(Conv1D(64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Conv1D(32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(SimpleRNN(64, return_sequences=True))
model.add(Dropout(0.2))
model.add(SimpleRNN(32, return_sequences=True))
model.add(Dropout(0.2))
model.add(MaxPooling1D(pool_size=3))
model.add(Dense(32, activation='tanh'))
model.add(GlobalMaxPooling1D())
model.add(Dense(11, activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


# train the model
model.fit(X_train_padded, y_train, epochs=15, batch_size=32)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 5777, 32)          1406304   
                                                                 
 conv1d_2 (Conv1D)           (None, 5777, 64)          6208      
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 1925, 64)         0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 1925, 32)          6176      
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 641, 32)          0         
 1D)                                                             
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 641, 64)          

In [18]:
tokenizer.fit_on_texts(X_test)
loss, acc = model.evaluate(X_test_padded, y_test)

60/60 [==============================] - 6s 96ms/step - loss: 1.6491 - accuracy: 0.6486


In [19]:
preds = model.predict(X_test_padded)

60/60 [==============================] - 6s 93ms/step


In [20]:
preds[0]

array([2.8149372e-03, 9.9429697e-01, 4.5813277e-11, 1.0549176e-04,
       2.7363331e-03, 4.9322665e-07, 1.5290649e-08, 4.5729081e-05,
       2.6765791e-08, 2.4716111e-18, 2.1701979e-13], dtype=float32)

In [21]:
preds_new = np.argmax(preds, axis = 1)

In [22]:
preds_new = np.array(preds_new)

In [23]:
preds_new[0]

1

In [24]:
from sklearn.metrics import f1_score,precision_score, recall_score, accuracy_score
f1 = f1_score(y_test, preds_new, average = 'macro')
print("F1 score = ",f1)
precision = precision_score(y_test, preds_new, average = 'macro')
recall = recall_score(y_test, preds_new, average = 'macro')
print("Precision = ", precision)
print("Recall = ", recall)
print("Accuracy = ", accuracy_score(y_test, preds_new))

F1 score =  0.6420385229243185
Precision =  0.6467518319811617
Recall =  0.6486292336253565
Accuracy =  0.6485774499473129


In [25]:
from sklearn.metrics import classification_report, roc_curve, auc, f1_score, accuracy_score, confusion_matrix
matrix = confusion_matrix(y_test, preds_new, labels = [0, 1, 2])
mat = matrix.diagonal()/matrix.sum(axis=1)
print(classification_report(y_test, preds_new, labels = [0, 1, 2],digits=4))
print('confusion matrix: ', mat)

              precision    recall  f1-score   support

           0     0.8160    0.8012    0.8085       166
           1     0.8883    0.8883    0.8883       179
           2     0.3750    0.3642    0.3695       173

   micro avg     0.6961    0.6853    0.6907       518
   macro avg     0.6931    0.6845    0.6888       518
weighted avg     0.6937    0.6853    0.6895       518

confusion matrix:  [0.89261745 0.95209581 0.96923077]
